# Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import glob
from pathlib import Path
from tqdm import tqdm
import json
import pickle
import gc
import time
import sys


In [146]:
def _delete_model(model):
    """Deletes the provided model"""
    gc.collect()
    torch.cuda.empty_cache()
    del model

# cuda implementation for faster processing.

In [147]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512,expandable_segments:True'

# Check CUDA availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Memory status function
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"Memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
        print(f"Memory cached: {torch.cuda.memory_cached()/1e9:.2f} GB")
        print(f"Max memory allocated: {torch.cuda.max_memory_allocated()/1e9:.2f} GB")


Using device: cuda


# Config

In [149]:
import os
import torch
import numpy as np
from pathlib import Path
import pandas as pd
from datetime import timedelta

def check_gpu_memory():
    """Monitor GPU memory usage"""
    if not torch.cuda.is_available():
        print("No GPU available!")
        return None

    total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    reserved = torch.cuda.memory_reserved(0) / (1024**3)
    allocated = torch.cuda.memory_allocated(0) / (1024**3)
    free = total - reserved

    print(f"\n{'='*50}")
    print(f"GPU Memory Status ({torch.cuda.get_device_name(0)})")
    print(f"{'='*50}")
    print(f"Total GPU Memory:     {total:.2f} GB")
    print(f"Reserved Memory:      {reserved:.2f} GB")
    print(f"Allocated Memory:     {allocated:.2f} GB")
    print(f"Free Memory:          {free:.2f} GB")
    print(f"Memory Utilization:   {(reserved/total)*100:.1f}%")
    print(f"{'='*50}")

    return {
        "total": round(total, 2),
        "reserved": round(reserved, 2),
        "allocated": round(allocated, 2),
        "free": round(free, 2),
        "utilization": round((reserved/total)*100, 1)
    }

class Config:
    # GPU Memory Management
    TOTAL_GPU_MEMORY = 39.56  # GB (from your output)
    MEMORY_RESERVE = 1.0      # GB (safety buffer)

    # Date ranges
    TRAIN_START_DATE = "2018-05-02T08:44:39.292059872Z"
    TRAIN_END_DATE = "2018-06-25T08:03:24.466039977Z"
    TEST_START_DATE = "2018-06-25T08:03:24.466039977Z"
    TEST_END_DATE = "2018-06-28T23:56:46.421875446Z"

    # Data files
    RESULTS_DIR = '/content/results'
    DATA_DIR = '/content/test'
    FILE_PREFIX = "xnas.itch_NVDA_"
    FILE_SUFFIX = ".csv"

    # Model Parameters
    BATCH_SIZE = 1000
    HIDDEN_SIZE = 256
    NUM_LAYERS = 4
    SEQUENCE_LENGTH = 1000
    PREDICTION_LENGTH = 300

    # Training Hyperparameters
    LEARNING_RATE = 0.0001
    EPOCHS = 100
    WARMUP_EPOCHS = 5
    WEIGHT_DECAY = 0.01
    GRADIENT_CLIP = 1.0

    # Performance Optimizations
    USE_AMP = True
    NUM_WORKERS = 0
    PIN_MEMORY = True
    PREFETCH_FACTOR = 2
    PERSISTENT_WORKERS = True
    GRADIENT_ACCUMULATION_STEPS = 1
    @classmethod
    def initialize(cls):
        """Initialize and validate configuration"""
        # Create directories
        os.makedirs(cls.RESULTS_DIR, exist_ok=True)
        os.makedirs(cls.DATA_DIR, exist_ok=True)

        # Initialize CUDA
        if torch.cuda.is_available():
            torch.cuda.empty_cache()  # Clear cache
            torch.cuda.set_device(0)  # Set primary GPU

            # Enable TF32 for A100 (faster with minimal precision loss)
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True

            # Set memory fraction
            usable_memory = (cls.TOTAL_GPU_MEMORY - cls.MEMORY_RESERVE) / cls.TOTAL_GPU_MEMORY
            torch.cuda.set_per_process_memory_fraction(usable_memory)

            # Print configuration
            print("\nModel Configuration:")
            print(f"Batch Size: {cls.BATCH_SIZE}")
            print(f"Hidden Size: {cls.HIDDEN_SIZE}")
            print(f"Number of Layers: {cls.NUM_LAYERS}")
            print(f"Sequence Length: {cls.SEQUENCE_LENGTH}")
            print(f"Learning Rate: {cls.LEARNING_RATE}")
            print(f"Using AMP: {cls.USE_AMP}")
            print(f"Number of Workers: {cls.NUM_WORKERS}")

            # Check memory usage
            memory_stats = check_gpu_memory()

            # Calculate theoretical model size
            model_size = cls.calculate_model_size()
            print(f"\nEstimated Model Size: {model_size:.2f} GB")

            if model_size > (cls.TOTAL_GPU_MEMORY - cls.MEMORY_RESERVE):
                print("\nWarning: Model might be too large for GPU memory!")
                return False

            return True
        return False


    @classmethod
    def calculate_model_size(cls):
        """Calculate approximate model memory usage in GB"""
        # LSTM parameters
        lstm_params = 4 * cls.HIDDEN_SIZE * (cls.HIDDEN_SIZE + 1) * cls.NUM_LAYERS

        # Linear layer parameters
        linear_params = cls.HIDDEN_SIZE * 1

        # Total parameters
        total_params = lstm_params + linear_params

        # Memory calculations (in bytes)
        param_memory = total_params * (2 if cls.USE_AMP else 4)  # Parameters
        optimizer_memory = total_params * 8  # Adam optimizer states

        # Batch memory
        batch_memory = (cls.BATCH_SIZE * cls.SEQUENCE_LENGTH *
                       cls.HIDDEN_SIZE * cls.NUM_LAYERS * 4)

        # Convert to GB
        total_memory = (param_memory + optimizer_memory + batch_memory) / (1024**3)

        return total_memory

    @classmethod
    def get_dataloader_kwargs(cls):
        """Get optimized DataLoader settings"""
        return {
            'batch_size': cls.BATCH_SIZE,
            'num_workers': cls.NUM_WORKERS,
            'pin_memory': cls.PIN_MEMORY,
            'prefetch_factor': cls.PREFETCH_FACTOR,
            'persistent_workers': cls.PERSISTENT_WORKERS,
            'drop_last': True
        }

    @classmethod
    def validate(cls):
        """Validate configuration settings"""
        model_size = cls.get_model_size()
        available_memory = cls.GPU_MEMORY - cls.MEMORY_RESERVE

        if model_size > available_memory:
            print(f"\nWarning: Estimated model memory ({model_size:.1f} GB) exceeds available GPU memory ({available_memory:.1f} GB)")
            print("Suggestions:")
            print("1. Reduce BATCH_SIZE")
            print("2. Reduce HIDDEN_SIZE")
            print("3. Reduce NUM_LAYERS")
            return False
        return True

    @classmethod
    def get_training_info(cls):
        """Get training configuration information"""
        model_size_gb = cls.get_model_size()
        batch_memory_gb = (cls.BATCH_SIZE * cls.SEQUENCE_LENGTH * 4) / (1024**3)

        print("\nTraining Configuration:")
        print(f"Estimated model size: {model_size_gb:.2f} GB")
        print(f"Estimated batch memory: {batch_memory_gb:.2f} GB")
        print(f"Using Automatic Mixed Precision: {cls.USE_AMP}")
        print(f"Batch size: {cls.BATCH_SIZE}")
        print(f"Hidden size: {cls.HIDDEN_SIZE}")
        print(f"Number of layers: {cls.NUM_LAYERS}")




    @classmethod
    def validate_dates(cls):
        try:
            train_start = pd.to_datetime(cls.TRAIN_START_DATE, utc=True)
            train_end = pd.to_datetime(cls.TRAIN_END_DATE, utc=True)
            test_start = pd.to_datetime(cls.TEST_START_DATE, utc=True)
            test_end = pd.to_datetime(cls.TEST_END_DATE, utc=True)

            print(f"\nValidating date ranges:")
            print(f"Train period: {train_start} to {train_end}")
            print(f"Test period: {test_start} to {test_end}")

            assert train_start < train_end, "Training start date must be before training end date"
            assert test_start < test_end, "Test start date must be before test end date"
            assert train_end <= test_start, "Training end date should be before or equal to test start date"

            # Adjust test_start if it's equal to train_end
            if train_end == test_start:
                cls.TEST_START_DATE = (test_start + pd.Timedelta(microseconds=1)).isoformat()
                print(f"Adjusted test start date to: {cls.TEST_START_DATE}")

            return True
        except Exception as e:
            print(f"Date validation error: {str(e)}")
            return False

    @classmethod
    def analyze_time_series(cls):
        try:
            all_diffs = []
            csv_files = glob.glob(str(Path(cls.DATA_DIR) / "*.csv"))

            for file in csv_files:
                df = pd.read_csv(file)
                df['ts_event'] = pd.to_datetime(df['ts_event'])
                df = df.sort_values('ts_event')
                time_diffs = df['ts_event'].diff().dt.total_seconds()
                all_diffs.extend(time_diffs.dropna().tolist())

            if not all_diffs:
                raise ValueError("No valid time differences found in the data")

            median_diff = np.median(all_diffs)
            mean_diff = np.mean(all_diffs)
            std_diff = np.std(all_diffs)

            typical_observations_per_30min = int((30 * 60) / median_diff)
            cls.SEQUENCE_LENGTH = min(max(typical_observations_per_30min, 10), 100)

            typical_observations_per_5min = int((5 * 60) / median_diff)
            cls.PREDICTION_LENGTH = min(max(typical_observations_per_5min, 5), 30)

            print(f"\nTime Series Analysis Results:")
            print(f"Median time between observations: {median_diff:.2f} seconds")
            print(f"Mean time between observations: {mean_diff:.2f} seconds")
            print(f"Standard deviation: {std_diff:.2f} seconds")
            print(f"Selected sequence length: {cls.SEQUENCE_LENGTH} observations")
            print(f"Selected prediction length: {cls.PREDICTION_LENGTH} observations")

            return True
        except Exception as e:
            print(f"Error analyzing time series: {str(e)}")
            return False

    @classmethod
    def get_file_list(cls):
        start_date = pd.to_datetime(cls.TRAIN_START_DATE).date()
        end_date = pd.to_datetime(cls.TEST_END_DATE).date()
        date_range = pd.date_range(start_date, end_date)

        file_list = []
        for date in date_range:
            file_name = f"{cls.FILE_PREFIX}{date.strftime('%Y%m%d')}_to_{(date + timedelta(days=1)).strftime('%Y%m%d')}{cls.FILE_SUFFIX}"
            file_path = os.path.join(cls.DATA_DIR, file_name)
            if os.path.exists(file_path):
                file_list.append(file_path)

        return file_list

    @classmethod
    def initialize(cls):
        if not cls.validate_dates():
            raise ValueError("Date validation failed")
        if not cls.analyze_time_series():
            print("Warning: Using default sequence and prediction lengths")
        return True


# Data Loading Function


In [150]:
def load_csv_data(file_list, columns=['ts_event', 'price']):
    dfs = []
    for file in file_list:
        df = pd.read_csv(file, usecols=columns)
        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)
    df['ts_event'] = pd.to_datetime(df['ts_event'], utc=True)
    return df.sort_values('ts_event')


In [151]:
def run_with_progress():
    # Initialize config
    Config.initialize()

    # Get file list
    file_list = Config.get_file_list()

    # Load data
    print("Loading data...")
    df = load_csv_data(file_list)

    # Preprocess data
    print("Preprocessing data...")
    train_start = pd.to_datetime(Config.TRAIN_START_DATE, utc=True)
    train_end = pd.to_datetime(Config.TRAIN_END_DATE, utc=True)
    test_start = pd.to_datetime(Config.TEST_START_DATE, utc=True)
    test_end = pd.to_datetime(Config.TEST_END_DATE, utc=True)

    X_train, y_train, X_test, y_test, scaler, test_df = preprocess_data(
        df, train_start, train_end, test_start, test_end, Config.SEQUENCE_LENGTH
    )

    # Create dataset and dataloader for training data
    train_dataset = PriceDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=Config.NUM_WORKERS)

    # Initialize model, loss function, and optimizer
    model = LSTMModel(hidden_size=Config.HIDDEN_SIZE, num_layers=Config.NUM_LAYERS)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)

    # Train the model
    print("Training model...")
    train_model_with_progress(model, train_loader, criterion, optimizer)

    # Make predictions on test data
    print("Making predictions...")
    test_predictions = predict_with_progress(model, X_test, scaler)

    # Create a DataFrame with test predictions
    predictions_df = pd.DataFrame({
        'ts_event': test_df['ts_event'][Config.SEQUENCE_LENGTH:].reset_index(drop=True),
        'predicted_price': test_predictions,
        'actual_price': test_df['price'][Config.SEQUENCE_LENGTH:].reset_index(drop=True)
    })

    # Plot predicted vs actual prices
    plot_predicted_vs_actual(predictions_df)

    print(predictions_df)

# Data Preprocessing Function


In [152]:
    def preprocess_data(self, df, train_start, train_end, test_start, test_end, sequence_length):
        """Preprocess data keeping larger datasets in CPU memory"""
        # Split data into train/test on CPU
        train_mask = (df['ts_event'] >= train_start) & (df['ts_event'] < train_end)
        test_mask = (df['ts_event'] >= test_start) & (df['ts_event'] <= test_end)

        train_df = df[train_mask]
        test_df = df[test_mask]

        # Scale data on CPU
        scaler = MinMaxScaler()
        train_prices = scaler.fit_transform(train_df['price'].values.reshape(-1, 1))
        test_prices = scaler.transform(test_df['price'].values.reshape(-1, 1))

        # Create sequences staying on CPU initially
        X_train, y_train = self._create_sequences(train_prices, sequence_length)
        X_test, y_test = self._create_sequences(test_prices, sequence_length)

        return X_train, y_train, X_test, y_test, scaler, test_df

    @staticmethod
    def _create_sequences(data, sequence_length):
        """Create sequences in CPU memory"""
        X, y = [], []
        for i in range(len(data) - sequence_length):
            X.append(data[i:i+sequence_length])
            y.append(data[i+sequence_length])
        return np.array(X), np.array(y)

In [153]:
class DataManager:
    def __init__(self, config):
        self.config = config

    def load_data(self, file_list, columns=['ts_event', 'price']):
        """Load data into CPU RAM"""
        # Load data in chunks to manage memory
        chunk_size = 1000000  # Adjust based on your RAM
        dfs = []
        for file in file_list:
            # Read CSV in chunks to control memory usage
            for chunk in pd.read_csv(file, usecols=columns, chunksize=chunk_size):
                dfs.append(chunk)

        df = pd.concat(dfs, ignore_index=True)
        df['ts_event'] = pd.to_datetime(df['ts_event'], utc=True)
        return df.sort_values('ts_event')

    def preprocess_data(self, df, train_start, train_end, test_start, test_end, sequence_length):
        """Preprocess data keeping larger datasets in CPU memory"""
        # Split data into train/test on CPU
        train_mask = (df['ts_event'] >= train_start) & (df['ts_event'] < train_end)
        test_mask = (df['ts_event'] >= test_start) & (df['ts_event'] <= test_end)

        train_df = df[train_mask]
        test_df = df[test_mask]

        # Scale data on CPU
        scaler = MinMaxScaler()
        train_prices = scaler.fit_transform(train_df['price'].values.reshape(-1, 1))
        test_prices = scaler.transform(test_df['price'].values.reshape(-1, 1))

        # Create sequences staying on CPU initially
        X_train, y_train = self.create_sequences(train_prices, sequence_length)
        X_test, y_test = self.create_sequences(test_prices, sequence_length)

        return X_train, y_train, X_test, y_test, scaler, test_df

    def create_sequences(self, data, sequence_length):
        """
        Create sequences for time series prediction.

        Args:
            data (numpy.ndarray): Input data array
            sequence_length (int): Length of each sequence

        Returns:
            tuple: Arrays of input sequences and target values
        """
        X, y = [], []

        # Process data in chunks to manage memory
        chunk_size = 10000
        total_sequences = len(data) - sequence_length

        for i in range(0, total_sequences, chunk_size):
            end_idx = min(i + chunk_size, total_sequences)

            # Create sequences for this chunk
            for j in range(i, end_idx):
                X.append(data[j:j+sequence_length])
                y.append(data[j+sequence_length])

            # Optional: Free memory if needed
            if i % (chunk_size * 10) == 0:
                torch.cuda.empty_cache()

        return np.array(X), np.array(y)

class BatchManager:
    def __init__(self, X, y, batch_size):
        """
        Initialize batch manager for memory-efficient training

        Args:
            X (numpy.ndarray): Input sequences
            y (numpy.ndarray): Target values
            batch_size (int): Size of each batch
        """
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.n_samples = len(X)
        self.current_idx = 0

    def __iter__(self):
        self.current_idx = 0
        # Create random indices for shuffling
        self.indices = np.random.permutation(self.n_samples)
        return self

    def __next__(self):
        if self.current_idx >= self.n_samples:
            raise StopIteration

        end_idx = min(self.current_idx + self.batch_size, self.n_samples)
        batch_indices = self.indices[self.current_idx:end_idx]

        # Move batch to GPU only when needed
        X_batch = torch.FloatTensor(self.X[batch_indices]).to(device)
        y_batch = torch.FloatTensor(self.y[batch_indices]).to(device)

        self.current_idx = end_idx
        return X_batch, y_batch

# Custom Dataset


In [154]:
class PriceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X).to(device)
        self.y = torch.FloatTensor(y).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# LSTM Model


In [155]:

class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Move states to same device as input
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward pass
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

    def save_model(self, path, optimizer=None, epoch=None, loss=None):
        """Save model state, optimizer state, and training info"""
        checkpoint = {
            'model_state_dict': self.state_dict(),
            'hidden_size': self.hidden_size,
            'num_layers': self.num_layers
        }

        if optimizer is not None:
            checkpoint['optimizer_state_dict'] = optimizer.state_dict()
        if epoch is not None:
            checkpoint['epoch'] = epoch
        if loss is not None:
            checkpoint['loss'] = loss

        torch.save(checkpoint, path)

    @classmethod
    def load_model(cls, path, device='cuda'):
        """Load a saved model"""
        checkpoint = torch.load(path, map_location=device)

        model = cls(
            hidden_size=checkpoint['hidden_size'],
            num_layers=checkpoint['num_layers']
        )

        model.load_state_dict(checkpoint['model_state_dict'])
        model = model.to(device)

        return model, checkpoint

In [156]:
class MemoryOptimizedDataset(Dataset):
    def __init__(self, data_dir, start_date, end_date, sequence_length, prediction_length):
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        self.data = self.load_data(data_dir, start_date, end_date)

    def load_data(self, data_dir, start_date, end_date):
        # Load data in chunks to avoid memory issues
        chunks = []
        for file in sorted(Path(data_dir).glob(f"{Config.FILE_PREFIX}*{Config.FILE_SUFFIX}")):
            try:
                chunk = pd.read_csv(file, parse_dates=['timestamp'])
                if chunk['timestamp'].min() <= end_date and chunk['timestamp'].max() >= start_date:
                    chunks.append(chunk)
            except Exception as e:
                print(f"Error loading {file}: {e}")

        data = pd.concat(chunks, ignore_index=True)
        data = data[(data['timestamp'] >= start_date) & (data['timestamp'] <= end_date)]
        return data.values  # Convert to numpy for memory efficiency

    def __len__(self):
        return max(0, len(self.data) - self.sequence_length - self.prediction_length)

    def __getitem__(self, idx):
        sequence = self.data[idx:idx + self.sequence_length]
        target = self.data[idx + self.sequence_length:idx + self.sequence_length + self.prediction_length]
        return torch.FloatTensor(sequence), torch.FloatTensor(target)

class MemoryOptimizedModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, sequence_length, prediction_length):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Memory-efficient architecture
        self.input_projection = nn.Linear(input_size, hidden_size)
        self.lstm_layers = nn.ModuleList([
            nn.LSTM(hidden_size, hidden_size, batch_first=True)
            for _ in range(num_layers)
        ])
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.output_projection = nn.Linear(hidden_size, prediction_length)

    def forward(self, x):
        # Input projection
        x = self.input_projection(x)

        # Process LSTM layers with checkpointing
        for lstm in self.lstm_layers:
            x, _ = lstm(x)
            x = self.layer_norm(x)
            x = self.dropout(x)

        # Output projection
        x = self.output_projection(x[:, -1])
        return x

def setup_memory_optimization():
    """Setup memory optimization configurations"""
    # Set PyTorch memory allocator settings
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

    # Enable CUDA memory optimization
    torch.cuda.empty_cache()
    gc.collect()

    if torch.cuda.is_available():
        # Calculate maximum batch size based on available memory
        available_memory = Config.TOTAL_GPU_MEMORY - Config.MEMORY_RESERVE
        print(f"Available GPU memory: {available_memory:.2f} GB")

        # Set CUDA memory optimization flags
        torch.backends.cudnn.benchmark = True
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

# Training Function


In [175]:
def train_model():
    """Main training function with memory optimization"""
    setup_memory_optimization()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize datasets
    train_dataset = MemoryOptimizedDataset(
        Config.DATA_DIR,
        Config.TRAIN_START_DATE,
        Config.TRAIN_END_DATE,
        Config.SEQUENCE_LENGTH,
        Config.PREDICTION_LENGTH
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        num_workers=Config.NUM_WORKERS,
        pin_memory=Config.PIN_MEMORY,
        prefetch_factor=Config.PREFETCH_FACTOR,
        persistent_workers=Config.PERSISTENT_WORKERS
    )

    # Initialize model
    input_size = train_dataset.data.shape[1]
    model = MemoryOptimizedModel(
        input_size=input_size,
        hidden_size=Config.HIDDEN_SIZE,
        num_layers=Config.NUM_LAYERS,
        sequence_length=Config.SEQUENCE_LENGTH,
        prediction_length=Config.PREDICTION_LENGTH
    ).to(device)

    # Initialize optimizer and scheduler
    optimizer = optim.AdamW(
        model.parameters(),
        lr=Config.LEARNING_RATE,
        weight_decay=Config.WEIGHT_DECAY
    )

    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=Config.LEARNING_RATE,
        epochs=Config.EPOCHS,
        steps_per_epoch=len(train_loader)
    )

    # Initialize AMP scaler
    scaler = GradScaler(enabled=Config.USE_AMP)

    # Training loop
    for epoch in range(Config.EPOCHS):
        model.train()
        total_loss = 0

        with tqdm(train_loader, desc=f'Epoch {epoch+1}/{Config.EPOCHS}') as pbar:
            for batch_idx, (data, target) in enumerate(pbar):
                data, target = data.to(device), target.to(device)

                # Clear gradients and cache
                optimizer.zero_grad(set_to_none=True)

                # Forward pass with AMP
                with autocast(enabled=Config.USE_AMP):
                    output = model(data)
                    loss = F.mse_loss(output, target)

                # Backward pass with gradient scaling
                scaler.scale(loss).backward()

                # Gradient clipping
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), Config.GRADIENT_CLIP)

                # Optimizer step with scaling
                scaler.step(optimizer)
                scaler.update()

                # Scheduler step
                scheduler.step()

                # Update progress bar
                total_loss += loss.item()
                pbar.set_postfix({'loss': total_loss / (batch_idx + 1)})

                # Memory cleanup
                if batch_idx % 10 == 0:
                    torch.cuda.empty_cache()

        # Save checkpoint
        if (epoch + 1) % 10 == 0:
            checkpoint_path = Path(Config.RESULTS_DIR) / f'checkpoint_epoch_{epoch+1}.pt'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': total_loss / len(train_loader),
            }, checkpoint_path)

    return model

# Prediction Function


In [158]:
def predict_with_progress(model, X_test, scaler, batch_size=64):
    """Make predictions with batched CPU to GPU transfer"""
    model.eval()
    predictions = []

    with torch.no_grad():
        # Process test data in batches
        for i in range(0, len(X_test), batch_size):
            batch = X_test[i:i+batch_size]
            X = torch.FloatTensor(batch).to(device)
            y_pred = model(X)
            predictions.extend(scaler.inverse_transform(y_pred.cpu().numpy()))

            # Clear GPU memory
            del X, y_pred
            torch.cuda.empty_cache()

    return predictions


# Visualization

In [177]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

def visualize_predictions(predictions_df):
    """
    Create comprehensive visualizations for price predictions

    Args:
        predictions_df: DataFrame with columns 'ts_event', 'actual_price', 'predicted_price'
    """
    # Set style
    plt.style.use('seaborn')

    # Calculate error metrics
    mse = mean_squared_error(predictions_df['actual_price'], predictions_df['predicted_price'])
    mae = mean_absolute_error(predictions_df['actual_price'], predictions_df['predicted_price'])
    r2 = r2_score(predictions_df['actual_price'], predictions_df['predicted_price'])

    # Calculate percentage error
    predictions_df['error'] = predictions_df['predicted_price'] - predictions_df['actual_price']
    predictions_df['error_pct'] = (predictions_df['error'] / predictions_df['actual_price']) * 100

    # Create subplots
    fig = plt.figure(figsize=(20, 15))

    # 1. Price Comparison Plot
    ax1 = plt.subplot(3, 1, 1)
    ax1.plot(predictions_df['ts_event'], predictions_df['actual_price'], label='Actual Price', color='blue', alpha=0.7)
    ax1.plot(predictions_df['ts_event'], predictions_df['predicted_price'], label='Predicted Price', color='red', alpha=0.7)
    ax1.set_title('Actual vs Predicted Price Over Time', fontsize=14, pad=20)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Price')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Error Distribution
    ax2 = plt.subplot(3, 2, 3)
    sns.histplot(predictions_df['error'], bins=50, ax=ax2, color='blue', alpha=0.6)
    ax2.set_title('Error Distribution', fontsize=12)
    ax2.set_xlabel('Error (Predicted - Actual)')
    ax2.set_ylabel('Count')

    # 3. Percentage Error Distribution
    ax3 = plt.subplot(3, 2, 4)
    sns.histplot(predictions_df['error_pct'], bins=50, ax=ax3, color='green', alpha=0.6)
    ax3.set_title('Percentage Error Distribution', fontsize=12)
    ax3.set_xlabel('Percentage Error')
    ax3.set_ylabel('Count')

    # 4. Scatter Plot
    ax4 = plt.subplot(3, 1, 3)
    ax4.scatter(predictions_df['actual_price'], predictions_df['predicted_price'],
               alpha=0.5, color='blue')

    # Add perfect prediction line
    min_val = min(predictions_df['actual_price'].min(), predictions_df['predicted_price'].min())
    max_val = max(predictions_df['actual_price'].max(), predictions_df['predicted_price'].max())
    ax4.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')

    ax4.set_title('Predicted vs Actual Prices', fontsize=12)
    ax4.set_xlabel('Actual Price')
    ax4.set_ylabel('Predicted Price')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    # Add text box with metrics
    metrics_text = f'Mean Squared Error: {mse:.4f}\nMean Absolute Error: {mae:.4f}\nR² Score: {r2:.4f}'
    plt.figtext(0.02, 0.02, metrics_text, fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Add main title
    plt.suptitle('Price Prediction Analysis', fontsize=16, y=0.98)

    return fig

def plot_error_analysis(predictions_df):
    """
    Create detailed error analysis visualizations

    Args:
        predictions_df: DataFrame with columns 'ts_event', 'actual_price', 'predicted_price'
    """
    # Calculate rolling metrics
    window_size = 100
    predictions_df['rolling_mse'] = (predictions_df['actual_price'] - predictions_df['predicted_price'])**2 \
                                   .rolling(window=window_size).mean()
    predictions_df['rolling_mae'] = abs(predictions_df['actual_price'] - predictions_df['predicted_price']) \
                                   .rolling(window=window_size).mean()

    # Create figure
    fig = plt.figure(figsize=(20, 10))

    # 1. Rolling Error Metrics
    ax1 = plt.subplot(2, 1, 1)
    ax1.plot(predictions_df['ts_event'], predictions_df['rolling_mse'],
             label=f'Rolling MSE (window={window_size})', color='red', alpha=0.7)
    ax1.plot(predictions_df['ts_event'], predictions_df['rolling_mae'],
             label=f'Rolling MAE (window={window_size})', color='blue', alpha=0.7)
    ax1.set_title('Rolling Error Metrics Over Time', fontsize=14)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Error')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Error Heatmap
    ax2 = plt.subplot(2, 1, 2)

    # Create price bins
    price_bins = pd.qcut(predictions_df['actual_price'], q=20)
    error_by_price = predictions_df.groupby(price_bins)['error'].agg(['mean', 'std']).round(4)

    sns.heatmap(error_by_price.T, annot=True, cmap='RdYlBu_r', ax=ax2, center=0)
    ax2.set_title('Error Analysis by Price Range', fontsize=14)
    ax2.set_xlabel('Price Range Percentile')

    plt.tight_layout()
    return fig

# Main execution


In [181]:
def prepare_data(config):
    """Prepare data with memory optimization"""
    print("Preparing data...")
    print_gpu_memory()

    config.initialize()
    file_list = config.get_file_list()

    # Load and preprocess data in chunks if needed
    df = load_csv_data(file_list)

    train_start = pd.to_datetime(config.TRAIN_START_DATE, utc=True)
    train_end = pd.to_datetime(config.TRAIN_END_DATE, utc=True)
    test_start = pd.to_datetime(config.TEST_START_DATE, utc=True)
    test_end = pd.to_datetime(config.TEST_END_DATE, utc=True)

    X_train, y_train, X_test, y_test, scaler, test_df = preprocess_data(
        df, train_start, train_end, test_start, test_end, config.SEQUENCE_LENGTH
    )

    # Clear memory
    del df
    torch.cuda.empty_cache()

    return X_train, y_train, X_test, y_test, scaler, test_df, (train_start, train_end, test_start, test_end)


In [162]:
def setup_training(X_train, y_train, config):
    """Setup training components including dataset, model and optimizer"""
    # Create dataset and dataloader
    train_dataset = PriceDataset(X_train, y_train)
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=config.NUM_WORKERS
    )

    # Initialize model, loss and optimizer
    model = LSTMModel(hidden_size=config.HIDDEN_SIZE, num_layers=config.NUM_LAYERS).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)

    return train_loader, model, criterion, optimizer

In [163]:
def train_and_predict(train_loader, model, criterion, optimizer, X_test, scaler, test_df, config):
    """Execute training and generate predictions"""
    # Train the model
    train_model_with_progress(model, train_loader, criterion, optimizer)

    # Generate predictions
    test_predictions = predict_with_progress(model, X_test, scaler)

    # Create predictions dataframe
    predictions_df = pd.DataFrame({
        'ts_event': test_df['ts_event'][config.SEQUENCE_LENGTH:].reset_index(drop=True),
        'predicted_price': test_predictions,
        'actual_price': test_df['price'][config.SEQUENCE_LENGTH:].reset_index(drop=True)
    })

    return predictions_df

In [ ]:
def run_pipeline():
    """Main pipeline with results saving and visualization"""
    try:
        print("Starting pipeline...")

        # Initialize data manager
        data_manager = DataManager(Config)
        file_list = Config.get_file_list()

        if not file_list:
            raise ValueError("No data files found in specified directory")

        df = data_manager.load_data(file_list)

        # Convert dates
        train_start = pd.to_datetime(Config.TRAIN_START_DATE, utc=True)
        train_end = pd.to_datetime(Config.TRAIN_END_DATE, utc=True)
        test_start = pd.to_datetime(Config.TEST_START_DATE, utc=True)
        test_end = pd.to_datetime(Config.TEST_END_DATE, utc=True)

        # Preprocess data
        X_train, y_train, X_test, y_test, scaler, test_df = data_manager.preprocess_data(
            df, train_start, train_end, test_start, test_end, Config.SEQUENCE_LENGTH
        )

        # Save scaler for future use
        scaler_path = os.path.join(Config.RESULTS_DIR, 'scaler.pkl')
        with open(scaler_path, 'wb') as f:
            pickle.dump(scaler, f)

        # Initialize model
        model = LSTMModel(hidden_size=Config.HIDDEN_SIZE, num_layers=Config.NUM_LAYERS).to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)

        # Train model
        train_model_with_progress(model, X_train, y_train, criterion, optimizer)

        # Generate predictions
        test_predictions = predict_with_progress(model, X_test, scaler)

        # Create and save predictions DataFrame
        predictions_df = pd.DataFrame({
            'ts_event': test_df['ts_event'][Config.SEQUENCE_LENGTH:].reset_index(drop=True),
            'predicted_price': test_predictions,
            'actual_price': test_df['price'][Config.SEQUENCE_LENGTH:].reset_index(drop=True)
        })

        # Save predictions
        predictions_path = os.path.join(Config.RESULTS_DIR, 'predictions.csv')
        predictions_df.to_csv(predictions_path, index=False)

        # Generate and save visualizations
        print("\nGenerating visualizations...")

        # Create main visualization
        fig1 = visualize_predictions(predictions_df)
        fig1_path = os.path.join(Config.RESULTS_DIR, 'prediction_analysis.png')
        fig1.savefig(fig1_path, dpi=300, bbox_inches='tight')
        plt.close(fig1)

        # Create error analysis visualization
        fig2 = plot_error_analysis(predictions_df)
        fig2_path = os.path.join(Config.RESULTS_DIR, 'error_analysis.png')
        fig2.savefig(fig2_path, dpi=300, bbox_inches='tight')
        plt.close(fig2)

        # Save model configuration
        config_path = os.path.join(Config.RESULTS_DIR, 'model_config.json')
        config_dict = {
            'HIDDEN_SIZE': Config.HIDDEN_SIZE,
            'NUM_LAYERS': Config.NUM_LAYERS,
            'SEQUENCE_LENGTH': Config.SEQUENCE_LENGTH,
            'LEARNING_RATE': Config.LEARNING_RATE,
            'TRAIN_START_DATE': Config.TRAIN_START_DATE,
            'TRAIN_END_DATE': Config.TRAIN_END_DATE,
            'TEST_START_DATE': Config.TEST_START_DATE,
            'TEST_END_DATE': Config.TEST_END_DATE
        }
        with open(config_path, 'w') as f:
            json.dump(config_dict, f, indent=4)

        print(f"\nResults saved in: {Config.RESULTS_DIR}")
        print(f"Model checkpoints saved in: {os.path.join(Config.RESULTS_DIR, 'models')}")
        print(f"Visualizations saved as:")
        print(f"- {fig1_path}")
        print(f"- {fig2_path}")

        # Display visualizations in notebook
        plt.figure(figsize=(20, 15))
        visualize_predictions(predictions_df)
        plt.show()

        plt.figure(figsize=(20, 10))
        plot_error_analysis(predictions_df)
        plt.show()

        return predictions_df

    except Exception as e:
        print(f"Error in pipeline: {str(e)}")
        raise e

if __name__ == "__main__":
    # Clear any existing cached memory
    torch.cuda.empty_cache()
    predictions_df = run_pipeline()

Starting pipeline...

Epoch [1/100]




  0%|          | 0/452 [00:00<?, ?batch/s]

  0%|          | 1/452 [00:00<03:14,  2.32batch/s]

  0%|          | 1/452 [00:00<03:14,  2.32batch/s, Loss=0.3762, Batch=1/452]

  0%|          | 2/452 [00:01<05:00,  1.50batch/s, Loss=0.3762, Batch=1/452]

  0%|          | 2/452 [00:01<05:00,  1.50batch/s, Loss=0.3772, Batch=2/452]

  1%|          | 3/452 [00:02<05:34,  1.34batch/s, Loss=0.3772, Batch=2/452]

  1%|          | 3/452 [00:02<05:34,  1.34batch/s, Loss=0.3708, Batch=3/452]

  1%|          | 4/452 [00:02<05:50,  1.28batch/s, Loss=0.3708, Batch=3/452]

  1%|          | 4/452 [00:02<05:50,  1.28batch/s, Loss=0.3660, Batch=4/452]

  1%|          | 5/452 [00:03<05:58,  1.25batch/s, Loss=0.3660, Batch=4/452]

  1%|          | 5/452 [00:03<05:58,  1.25batch/s, Loss=0.3619, Batch=5/452]

  1%|▏         | 6/452 [00:04<06:03,  1.23batch/s, Loss=0.3619, Batch=5/452]

  1%|▏         | 6/452 [00:04<06:03,  1.23batch/s, Loss=0.3567, Batch=6/452]

  2%|▏         | 7/452 [00:05<06:06,  1.21ba

Epoch 1 - Average Loss: 0.0214

Epoch [2/100]


 90%|█████████ | 409/452 [05:25<00:34,  1.26batch/s, Loss=0.0000, Batch=409/452]

In [180]:
import torch

def check_gpu_memory():
    """
    Check and display detailed GPU memory usage.

    Returns:
        dict: Memory statistics in GB
    """
    if not torch.cuda.is_available():
        print("No GPU available!")
        return None

    # Get memory statistics in GB
    total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    reserved = torch.cuda.memory_reserved(0) / (1024**3)
    allocated = torch.cuda.memory_allocated(0) / (1024**3)
    free = total - reserved

    # Print memory status
    print(f"\n{'='*50}")
    print(f"GPU Memory Status ({torch.cuda.get_device_name(0)})")
    print(f"{'='*50}")
    print(f"Total GPU Memory:     {total:.2f} GB")
    print(f"Reserved Memory:      {reserved:.2f} GB")
    print(f"Allocated Memory:     {allocated:.2f} GB")
    print(f"Free Memory:          {free:.2f} GB")
    print(f"Memory Utilization:   {(reserved/total)*100:.1f}%")
    print(f"{'='*50}")

    return {
        "total": round(total, 2),
        "reserved": round(reserved, 2),
        "allocated": round(allocated, 2),
        "free": round(free, 2),
        "utilization": round((reserved/total)*100, 1)
    }

# Actually run the function to see the output
memory_stats = check_gpu_memory()


GPU Memory Status (NVIDIA A100-SXM4-40GB)
Total GPU Memory:     39.56 GB
Reserved Memory:      1.75 GB
Allocated Memory:     1.73 GB
Free Memory:          37.81 GB
Memory Utilization:   4.4%


In [179]:
import torch
import gc
import os
import logging
from typing import Optional, Dict

class GPUMemoryManager:
    def __init__(self, threshold_gb: float = 5.0, debug: bool = True):
        """
        Initialize the GPU Memory Manager

        Args:
            threshold_gb (float): Minimum free memory threshold in GB to trigger cleanup
            debug (bool): Whether to print detailed memory information
        """
        self.threshold_gb = threshold_gb
        self.debug = debug
        self.setup_logging()

        # Set PyTorch memory allocation settings
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = (
            'max_split_size_mb:512,'
            'expandable_segments:True,'
            'garbage_collection_threshold:0.8'
        )

    def setup_logging(self):
        """Setup logging configuration"""
        logging.basicConfig(
            level=logging.DEBUG if self.debug else logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def get_gpu_memory_info(self) -> Dict[str, float]:
        """
        Get detailed GPU memory information using PyTorch

        Returns:
            Dict containing memory information in GB
        """
        if not torch.cuda.is_available():
            return {"error": "CUDA not available"}

        try:
            # Get memory information from PyTorch
            allocated = torch.cuda.memory_allocated() / 1e9  # Convert to GB
            reserved = torch.cuda.memory_reserved() / 1e9
            max_memory = torch.cuda.max_memory_allocated() / 1e9

            # Calculate total GPU memory
            total = torch.cuda.get_device_properties(0).total_memory / 1e9
            free = total - allocated

            return {
                "total_memory_gb": total,
                "free_memory_gb": free,
                "used_memory_gb": allocated,
                "allocated_memory_gb": allocated,
                "reserved_memory_gb": reserved,
                "max_allocated_gb": max_memory,
            }
        except Exception as e:
            self.logger.error(f"Error getting GPU memory info: {str(e)}")
            return {"error": str(e)}

    def print_memory_stats(self):
        """Print detailed memory statistics"""
        memory_info = self.get_gpu_memory_info()

        if "error" in memory_info:
            self.logger.error(f"Could not get memory stats: {memory_info['error']}")
            return

        self.logger.info("\n=== GPU Memory Statistics ===")
        self.logger.info(f"Total GPU Memory: {memory_info['total_memory_gb']:.2f} GB")
        self.logger.info(f"Free GPU Memory: {memory_info['free_memory_gb']:.2f} GB")
        self.logger.info(f"Used GPU Memory: {memory_info['used_memory_gb']:.2f} GB")
        self.logger.info(f"Allocated by PyTorch: {memory_info['allocated_memory_gb']:.2f} GB")
        self.logger.info(f"Reserved by PyTorch: {memory_info['reserved_memory_gb']:.2f} GB")
        self.logger.info(f"Max Allocated: {memory_info['max_allocated_gb']:.2f} GB")
        self.logger.info("===========================\n")

    def clear_memory(self, force: bool = False) -> bool:
        """
        Clear GPU memory if usage is above threshold or if forced

        Args:
            force (bool): Force memory cleanup regardless of threshold

        Returns:
            bool: True if cleanup was performed, False otherwise
        """
        if not torch.cuda.is_available():
            self.logger.warning("CUDA not available")
            return False

        memory_info = self.get_gpu_memory_info()

        if "error" in memory_info:
            self.logger.error(f"Could not clear memory: {memory_info['error']}")
            return False

        if force or memory_info['free_memory_gb'] < self.threshold_gb:
            try:
                # Print initial state if debugging
                if self.debug:
                    self.logger.info("Before cleanup:")
                    self.print_memory_stats()

                # Clear CUDA cache
                torch.cuda.empty_cache()

                # Run garbage collector
                gc.collect()

                # Clear peak memory stats
                torch.cuda.reset_peak_memory_stats()
                if hasattr(torch.cuda, 'reset_accumulated_memory_stats'):
                    torch.cuda.reset_accumulated_memory_stats()

                # Delete any remaining CUDA tensors
                for obj in gc.get_objects():
                    try:
                        if torch.is_tensor(obj):
                            if obj.is_cuda:
                                del obj
                    except Exception:
                        pass

                # Run garbage collector again
                gc.collect()
                torch.cuda.empty_cache()

                # Print final state if debugging
                if self.debug:
                    self.logger.info("After cleanup:")
                    self.print_memory_stats()

                return True

            except Exception as e:
                self.logger.error(f"Error during memory cleanup: {str(e)}")
                return False

        return False

    def get_memory_fragmentation(self) -> Optional[float]:
        """
        Calculate memory fragmentation ratio

        Returns:
            float: Fragmentation ratio (0-1) or None if calculation fails
        """
        try:
            memory_info = self.get_gpu_memory_info()
            if "error" in memory_info:
                return None

            allocated = memory_info['allocated_memory_gb']
            reserved = memory_info['reserved_memory_gb']

            if reserved == 0:
                return 0.0

            return 1.0 - (allocated / reserved)
        except Exception as e:
            self.logger.error(f"Error calculating fragmentation: {str(e)}")
            return None

    def is_memory_critical(self) -> bool:
        """
        Check if memory usage is in a critical state

        Returns:
            bool: True if memory usage is critical
        """
        memory_info = self.get_gpu_memory_info()

        if "error" in memory_info:
            return True

        free_memory = memory_info['free_memory_gb']
        total_memory = memory_info['total_memory_gb']

        # Consider memory critical if less than 10% free
        return free_memory < (total_memory * 0.1)

def quick_memory_cleanup():
    """
    Quick function to clear GPU memory without creating a manager instance
    """
    if torch.cuda.is_available():
        # Clear CUDA cache
        torch.cuda.empty_cache()

        # Run garbage collector
        gc.collect()

        # Reset memory stats
        torch.cuda.reset_peak_memory_stats()

        # Delete CUDA tensors
        for obj in gc.get_objects():
            try:
                if torch.is_tensor(obj):
                    if obj.is_cuda:
                        del obj
            except Exception:
                pass

        # Final cleanup
        gc.collect()
        torch.cuda.empty_cache()

# Example usage
if __name__ == "__main__":
    # Option 1: Using the full manager
    memory_manager = GPUMemoryManager(threshold_gb=5.0, debug=True)
    memory_manager.print_memory_stats()
    memory_manager.clear_memory(force=True)

    # Option 2: Quick cleanup
    quick_memory_cleanup()

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1022: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)


if __name__ == "__main__":
    # Initialize config
    Config.initialize()

    # Get file list
    file_list = Config.get_file_list()

    # Load data
    df = load_csv_data(file_list)

    # Preprocess data
    train_start = pd.to_datetime(Config.TRAIN_START_DATE, utc=True)
    train_end = pd.to_datetime(Config.TRAIN_END_DATE, utc=True)
    test_start = pd.to_datetime(Config.TEST_START_DATE, utc=True)
    test_end = pd.to_datetime(Config.TEST_END_DATE, utc=True)

    # Plot training data
    plot_train_data(df, train_start, train_end)

    X_train, y_train, X_test, y_test, scaler, test_df = preprocess_data(
        df, train_start, train_end, test_start, test_end, Config.SEQUENCE_LENGTH
    )

    # Create dataset and dataloader for training data
    train_dataset = PriceDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=Config.NUM_WORKERS)
    
    # Initialize model, loss function, and optimizer
    model = LSTMModel(hidden_size=Config.HIDDEN_SIZE, num_layers=Config.NUM_LAYERS)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)
    
    # Train the model
    train_model_with_progress(model, train_loader, criterion, optimizer)
    
    # Make predictions on test data
    test_predictions = predict_with_progress(model, X_test, scaler)
    
    # Create a DataFrame with test predictions
    predictions_df = pd.DataFrame({
        'ts_event': test_df['ts_event'][Config.SEQUENCE_LENGTH:].reset_index(drop=True),
        'predicted_price': test_predictions,
        'actual_price': test_df['price'][Config.SEQUENCE_LENGTH:].reset_index(drop=True)
    })
    
    # Plot predicted vs actual prices for test data
    plot_predicted_vs_actual(predictions_df, test_start, test_end)
    
    print(predictions_df)
